# WK3

## 1. Download and Explore Dataset

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

!conda install -c anaconda beautifulsoup4 -y

from urllib.request import urlopen
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following pa

In [3]:
import requests 
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

In [23]:
soup = BeautifulSoup(page.text, 'html.parser')

results = soup.find("table", {"class": "wikitable"})
#print(results.prettify())
table_rows = results.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df = pd.DataFrame(res, columns=["postal_code", "borough", "neighbourhood"])
df.replace("Not assigned", np.nan, inplace = True)
df.dropna(subset=["borough", "neighbourhood"], how='all', inplace=True)
df.drop(columns=['postal_code'], axis=1, inplace=True)
print(df)

              borough                                      neighbourhood
2          North York                                          Parkwoods
3          North York                                   Victoria Village
4    Downtown Toronto                          Regent Park, Harbourfront
5          North York                   Lawrence Manor, Lawrence Heights
6    Downtown Toronto        Queen's Park, Ontario Provincial Government
8           Etobicoke            Islington Avenue, Humber Valley Village
9         Scarborough                                     Malvern, Rouge
11         North York                                          Don Mills
12          East York                    Parkview Hill, Woodbine Gardens
13   Downtown Toronto                           Garden District, Ryerson
14         North York                                          Glencairn
17          Etobicoke  West Deane Park, Princess Gardens, Martin Grov...
18        Scarborough             Rouge Hill, Port 

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [66]:
#Split the comma separated value into multiple rows and remove duplicate.

new_df = pd.DataFrame(df.neighbourhood.str.split(',').tolist(), index=df.borough).stack()

new_df=new_df.reset_index([0, 'borough'])
new_df.columns = ['borough', 'neighbourhood']# Result
new_df['neighbourhood'] = new_df['neighbourhood'].str.strip()
new_df = new_df.drop_duplicates(subset=['borough', 'neighbourhood'])
new_df.head(20)

new_df.count()
neighborhoods = new_df['neighbourhood']
neighborhoods.nunique()
neighborhoods

,borough,neighbourhood
0,North York,Parkwoods
1,North York,Victoria Village
2,Downtown Toronto,Regent Park
3,Downtown Toronto,Harbourfront
4,North York,Lawrence Manor
5,North York,Lawrence Heights
6,Downtown Toronto,Queen's Park
7,Downtown Toronto,Ontario Provincial Government
8,Etobicoke,Islington Avenue
9,Etobicoke,Humber Valley Village


In [67]:
#Address tester
address = 'Keelsdale and Silverthorn'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address + ', Toronto')
print(location)
if location is None:
    location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

None


AttributeError: 'NoneType' object has no attribute 'latitude'

In [101]:
#Collect coordinate and merge back to df
latitude = [] 
longitude = [] 

for index, row in new_df.iterrows(): 
    location = geolocator.geocode(row['neighbourhood'] + ', Toronto') #More accurate
    if location is None:
        location = geolocator.geocode(row['neighbourhood']) #Handle exception case
    #print('The geograpical coordinate of {}.'.format(row['neighbourhood']))
    if location is not None:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        print('The geograpical coordinate of {} are {}, {}.'.format(row['neighbourhood'], location.latitude, location.longitude))
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)
new_df['latitude'] = latitude
new_df['longitude'] = longitude

The geograpical coordinate of Parkwoods are 43.7587999, -79.3201966.
The geograpical coordinate of Victoria Village are 43.732658, -79.3111892.
The geograpical coordinate of Regent Park are 43.6607056, -79.3604569.
The geograpical coordinate of Harbourfront are 43.6400801, -79.3801495.
The geograpical coordinate of Lawrence Manor are 43.7220788, -79.4375067.
The geograpical coordinate of Lawrence Heights are 43.7227784, -79.4509332.
The geograpical coordinate of Queen's Park are 43.659659, -79.3903399.
The geograpical coordinate of Ontario Provincial Government are 46.4291226, -80.9520897.
The geograpical coordinate of Islington Avenue are 43.6389593, -79.5210499.
The geograpical coordinate of Humber Valley Village are 43.6664717, -79.5243136.
The geograpical coordinate of Malvern are 43.8091955, -79.2217008.
The geograpical coordinate of Rouge are 43.8049304, -79.1658374.
The geograpical coordinate of Don Mills are 43.775347, -79.3459439.
The geograpical coordinate of Parkview Hill ar

In [102]:
#Clean up NaN
new_df.dropna(subset=["latitude"], axis=0, how='all', inplace=True)
new_df

,borough,neighbourhood,latitude,longitude
0,North York,Parkwoods,43.758800,-79.320197
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Regent Park,43.660706,-79.360457
3,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,North York,Lawrence Manor,43.722079,-79.437507
5,North York,Lawrence Heights,43.722778,-79.450933
6,Downtown Toronto,Queen's Park,43.659659,-79.390340
7,Downtown Toronto,Ontario Provincial Government,46.429123,-80.952090
8,Etobicoke,Islington Avenue,43.638959,-79.521050
9,Etobicoke,Humber Valley Village,43.666472,-79.524314


In [107]:
#Get coordinate for Toronto
address = 'Toronto, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


In [108]:
#Toronto and neighbourhood

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(new_df['latitude'], new_df['longitude'], new_df['borough'], new_df['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    #print(str(lat) +', ' + str(lng))
    folium.CircleMarker(   
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
    
map_toronto

In [110]:
#Only get Downtown and neighbour
df_downtown = new_df[new_df['borough'] == 'Downtown Toronto'].reset_index(drop=True)
df_downtown.head()

,borough,neighbourhood,latitude,longitude
0,Downtown Toronto,Regent Park,43.660706,-79.360457
1,Downtown Toronto,Harbourfront,43.640080,-79.380150
2,Downtown Toronto,Queen's Park,43.659659,-79.390340
3,Downtown Toronto,Ontario Provincial Government,46.429123,-80.952090
4,Downtown Toronto,Garden District,43.656500,-79.377114


In [113]:
#Get coordinate for Downtown Tornoto
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [116]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_downtown['latitude'], df_downtown['longitude'], df_downtown['neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [121]:
CLIENT_ID = 'GMEKW1WHK5ZTE2K5K3KI4CHSFWGGXTRY41YMYKSGHXRVKA43' # your Foursquare ID
CLIENT_SECRET = 'M3IHRIYPKEZWRJVU0NJWKRMJYB5U2GYZWNRYAY010AEKEHE4' # your Foursquare Secret
ACCESS_TOKEN = '3WWMQECCLLL4VLRHZNUT0M04FMKFLT5P04IVFCGCWHJNK0NV' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GMEKW1WHK5ZTE2K5K3KI4CHSFWGGXTRY41YMYKSGHXRVKA43
CLIENT_SECRET:M3IHRIYPKEZWRJVU0NJWKRMJYB5U2GYZWNRYAY010AEKEHE4


In [119]:
df_downtown.loc[0, 'neighbourhood']
neighbourhood_latitude = df_downtown.loc[0, 'latitude'] # neighborhood latitude value
neighbourhood_longitude = df_downtown.loc[0, 'longitude'] # neighborhood longitude value

neighbourhood_name = df_downtown.loc[0, 'neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park are 43.6607056, -79.3604569.


In [122]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GMEKW1WHK5ZTE2K5K3KI4CHSFWGGXTRY41YMYKSGHXRVKA43&client_secret=M3IHRIYPKEZWRJVU0NJWKRMJYB5U2GYZWNRYAY010AEKEHE4&v=20180605&ll=43.6607056,-79.3604569&radius=500&limit=100'

In [123]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '603a8ed450c1fe01f3c2f350'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Regent Park',
  'headerFullLocation': 'Regent Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 43.6652056045, 'lng': -79.35424823370592},
   'sw': {'lat': 43.6562055955, 'lng': -79.36666556629409}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5092fe87e4b0476c6e7375e2',
       'name': 'Regent Park Aquatic Centre',
       'location': {'address': '640 Dundas St. East',
        'crossStreet': 'at Sumach St.',
        'lat': 43.6606003130796,
        'lng': -79.36139162915349,
        'labeledLatLngs'

In [124]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [125]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Sumach Espresso,Coffee Shop,43.658135,-79.359515
2,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808
3,Thai To Go,Thai Restaurant,43.663418,-79.360710
4,Paintbox Bistro,Restaurant,43.660050,-79.362855


In [126]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Downtown

In [127]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [129]:

downtown_venues = getNearbyVenues(names=df_downtown['neighbourhood'],
                                   latitudes=df_downtown['latitude'],
                                   longitudes=df_downtown['longitude']
                                  )
print(downtown_venues.shape)
downtown_venues.head()

Regent Park
Harbourfront
Queen's Park
Ontario Provincial Government
Garden District
Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Harbourfront East
Union Station
Toronto Islands
Toronto Dominion Centre
Design Exchange
Commerce Court
Victoria Hotel
University of Toronto
Harbord
Kensington Market
Chinatown
Grange Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
Stn A PO Boxes
Cabbagetown
First Canadian Place
Underground city
Church and Wellesley
(2314, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,Regent Park,43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
2,Regent Park,43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
3,Regent Park,43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant
4,Regent Park,43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant


In [131]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,26,26,26,26,26,26
Berczy Park,100,100,100,100,100,100
CN Tower,54,54,54,54,54,54
Cabbagetown,47,47,47,47,47,47
Central Bay Street,61,61,61,61,61,61
Chinatown,64,64,64,64,64,64
Christie,58,58,58,58,58,58
Church and Wellesley,30,30,30,30,30,30


In [133]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 234 uniques categories.


## 3. Analyze Each Neighborhood

In [134]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [135]:
downtown_onehot.shape

(2314, 234)

In [136]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.040000,0.000000,0.070000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.020000,0.00,0.010000,0.00,0.000000,0.020000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [137]:
downtown_grouped.shape

(38, 234)

In [138]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2           Restaurant  0.06
3                  Gym  0.05
4  Japanese Restaurant  0.04


----Bathurst Quay----
              venue  freq
0       Coffee Shop  0.15
1              Café  0.12
2              Park  0.08
3     Grocery Store  0.08
4  Sushi Restaurant  0.04


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Restaurant  0.05
3  Japanese Restaurant  0.04
4   Italian Restaurant  0.04


----CN Tower----
              venue  freq
0             Hotel  0.11
1       Coffee Shop  0.07
2  Baseball Stadium  0.06
3       Pizza Place  0.06
4          Aquarium  0.04


----Cabbagetown----
                venue  freq
0   Indian Restaurant  0.06
1                Café  0.06
2         Coffee Shop  0.06
3          Restaurant  0.06
4  Italian Restaurant  0.04


----Central Bay Street----
                venue  freq
0         C

In [139]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [163]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Gym,Hotel,Japanese Restaurant,Italian Restaurant,Clothing Store,American Restaurant,Gastropub
1,Bathurst Quay,Coffee Shop,Café,Park,Grocery Store,Caribbean Restaurant,Ramen Restaurant,Garden,Japanese Restaurant,Diner,Bank
2,Berczy Park,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Japanese Restaurant,Gym,Gastropub,Beer Bar
3,CN Tower,Hotel,Coffee Shop,Baseball Stadium,Pizza Place,Gym,Scenic Lookout,Ice Cream Shop,Bar,Aquarium,Sandwich Place
4,Cabbagetown,Indian Restaurant,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Diner,Pub,Gastropub,Beer Store


## 4. Cluster Neighborhoods

In [164]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 3, 0, 0], dtype=int32)

In [165]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_downtown
downtown_merged.rename(columns={'neighbourhood': 'Neighborhood'}, inplace=True)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Regent Park,43.660706,-79.360457,0,Coffee Shop,Thai Restaurant,Restaurant,Fast Food Restaurant,Food Truck,Pub,Beer Store,Pool,Intersection,Electronics Store
1,Downtown Toronto,Harbourfront,43.640080,-79.380150,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Plaza,Sporting Goods Shop,Sports Bar,Steakhouse,Pizza Place
2,Downtown Toronto,Queen's Park,43.659659,-79.390340,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Japanese Restaurant,Bank,Bubble Tea Shop,Juice Bar,French Restaurant,Thai Restaurant
3,Downtown Toronto,Ontario Provincial Government,46.429123,-80.952090,2,Tree,Women's Store,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,Downtown Toronto,Garden District,43.656500,-79.377114,0,Clothing Store,Hotel,Coffee Shop,Sandwich Place,Café,Lingerie Store,Theater,Pizza Place,Japanese Restaurant,Cosmetics Shop


In [167]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['latitude'], downtown_merged['longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters